In [1]:
import requests
import pandas as pd
import datetime

In [3]:
data = pd.DataFrame(columns=['latitude', 'datacoverage', 'longitude', 'name', 'maxdate', 'id', 'elevationUnit', 'elevation', 'mindate'])

In [4]:
headers = {'token': 'RNxQMNDKxTaZpTvGYgBiwvluqyPswRTi'}


r = requests.get('http://www.ncdc.noaa.gov/cdo-web/api/v2/stations?limit=1000&datatypeid=SNWD&extent=35.81781315869664,-12.3046875,60.23981116999892,42.890625', headers=headers)
for result in r.json()['results']:
    data = data.append(result, ignore_index=True)

In [13]:
stations = {}

for row in data.iterrows():
    if row[1]['name'].endswith('EZ'):
        stations[row[1]['id']] = row[1]['name']

In [43]:
data[data.name.str.endswith('EZ')].to_csv('data/stations.csv', encoding='utf-8')

In [14]:
stations

{'GHCND:EZ000011464': 'MILESOVKA, EZ',
 'GHCND:EZM00011406': 'CHEB, EZ',
 'GHCND:EZM00011423': 'PRIMDA, EZ',
 'GHCND:EZM00011487': 'KOCELOVICE, EZ',
 'GHCND:EZM00011518': 'RUZYNE, EZ',
 'GHCND:EZM00011520': 'PRAHA LIBUS, EZ',
 'GHCND:EZM00011603': 'LIBEREC, EZ',
 'GHCND:EZM00011659': 'PRIBYSLAV, EZ',
 'GHCND:EZM00011723': 'TURANY, EZ',
 'GHCND:EZM00011782': 'MOSNOV, EZ',
 'GHCND:EZM00011787': 'LYSA HORA, EZ'}

In [31]:
out = {}

In [32]:
for station in stations:
    for year in range(1970, 2015):
        headers = {'token': 'RNxQMNDKxTaZpTvGYgBiwvluqyPswRTi'}
        r = requests.get('http://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=SNWD&stationid=' + station + '&startdate=' + str(year) + '-12-24&enddate=' + str(year) + '-12-24&limit=1000', headers=headers)
        try:
            for result in r.json()['results']:
                if(result['date'] not in out):
                    out[result['date']] = {}
                out[result['date']][stations[station]] = result['value']
            print(stations[station])
        except:
            print('!', stations[station])

! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
MOSNOV, EZ
MOSNOV, EZ
! MOSNOV, EZ
! MOSNOV, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
! KOCELOVICE, EZ
KOCELOVICE, EZ
! KOCELOVICE, EZ
KOC

In [34]:
outd = pd.DataFrame.from_dict(out, orient='index')

In [42]:
outd.transpose().to_csv('data/snowdepth.csv', encoding='utf-8')

In [3]:
#grouping atd
snih = pd.read_csv('data/snowdepth.csv')
stanice = pd.read_csv('data/stations.csv')

In [44]:
snih.fillna(0, inplace=True)

In [46]:
def prumerator(row):
    ano = 0
    for col in snih.columns:
        if (col == 'Unnamed: 0'):
            continue
        if (row[col] != 0):
            ano += 1
    return ((ano / (len(snih.columns) - 1)) * 100)

In [47]:
snih['pct'] = snih.apply(lambda row: prumerator(row), 1)

In [49]:
merged = stanice.merge(snih, left_on='name', right_on='Unnamed: 0', how='right')

In [52]:
merged.columns

Index(['Unnamed: 0_x', 'latitude', 'datacoverage', 'longitude', 'name',
       'maxdate', 'id', 'elevationUnit', 'elevation', 'mindate',
       'Unnamed: 0_y', '1982-12-24T00:00:00', '1983-12-24T00:00:00',
       '1984-12-24T00:00:00', '1985-12-24T00:00:00', '1986-12-24T00:00:00',
       '1987-12-24T00:00:00', '1988-12-24T00:00:00', '1989-12-24T00:00:00',
       '1990-12-24T00:00:00', '1991-12-24T00:00:00', '1992-12-24T00:00:00',
       '1993-12-24T00:00:00', '1994-12-24T00:00:00', '1995-12-24T00:00:00',
       '1996-12-24T00:00:00', '1997-12-24T00:00:00', '1998-12-24T00:00:00',
       '1999-12-24T00:00:00', '2000-12-24T00:00:00', '2001-12-24T00:00:00',
       '2002-12-24T00:00:00', '2003-12-24T00:00:00', '2004-12-24T00:00:00',
       '2005-12-24T00:00:00', '2006-12-24T00:00:00', '2007-12-24T00:00:00',
       '2008-12-24T00:00:00', '2009-12-24T00:00:00', '2010-12-24T00:00:00',
       '2011-12-24T00:00:00', '2012-12-24T00:00:00', '2013-12-24T00:00:00',
       'pct'],
      dtype='object

In [54]:
#rename
for col in merged.columns:
    if(col.endswith('T00:00:00')):
        merged.rename(columns={col: col.split('-')[0]}, inplace=True)

In [55]:
merged.columns

Index(['Unnamed: 0_x', 'latitude', 'datacoverage', 'longitude', 'name',
       'maxdate', 'id', 'elevationUnit', 'elevation', 'mindate',
       'Unnamed: 0_y', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', 'pct'],
      dtype='object')

In [56]:
merged.to_csv('data/summarized.csv', encoding='utf-8')